In [3]:
import findspark as fs

import os
import sys
import numpy as np
import pandas as pd

print(sys.executable)
print(sys.path)
print(sys.version)

import photutils
from photutils import detect_threshold
from astropy.io import fits
from astropy.wcs import WCS
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.modeling.functional_models import Gaussian2D

#from pyspark.sql.types import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession

from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from photutils import detect_sources
import time

import shutil
from math import pi, log, sqrt

import warnings
warnings.filterwarnings("ignore")

/home/s2494876/miniconda3/envs/sddm/bin/python
['/home/s2494876/miniconda3/envs/sddm/lib/python36.zip', '/home/s2494876/miniconda3/envs/sddm/lib/python3.6', '/home/s2494876/miniconda3/envs/sddm/lib/python3.6/lib-dynload', '', '/home/s2494876/.local/lib/python3.6/site-packages', '/home/s2494876/miniconda3/envs/sddm/lib/python3.6/site-packages', '/home/s2494876/.local/lib/python3.6/site-packages/IPython/extensions', '/home/s2494876/.ipython']
3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 23:51:54) 
[GCC 7.3.0]


In [34]:
# Define data paths
DR2_path = "/data/astronomy-big-data/bc96e9620e41b9aba98292d37b5eec24/LoTSS_DR2/"
mosaic_path = "/data/astronomy-big-data/bc96e9620e41b9aba98292d37b5eec24/LoTSS_DR2_mosaic/"
writeable = "/data/astronomy-big-data/bc96e9620e41b9aba98292d37b5eec24/LoTSS_DR2_writable/"

In [45]:
radius = pd.read_csv("overlap_dict.csv", sep=";", header=0, usecols=["fitsfile", "radius"])
radius['radius'] *= 1
radius.head()

,fitsfile,radius
0,P22Hetdex04_mosaic-blanked.fits,8.579995
1,P22Hetdex04_mosaic-blanked.fits,7.229981
2,P22Hetdex04_mosaic-blanked.fits,7.933324
3,P22Hetdex04_mosaic-blanked.fits,6.427702
4,P22Hetdex04_mosaic-blanked.fits,6.303318


In [40]:
header = ["label", "total_pixels", "x_pixels", "y_pixels",
          "integrated_intensity", "brightest_pixel", "brightest_pixel_x", "brightest_pixel_y",
          "brightest_pixel_RA", "brightest_pixel_DEC", "center_of_mass_x", "center_of_mass_y",
          "center_of_mass_RA", "center_of_mass_DEC", "center_of_gaus_fit_x", "center_of_gaus_fit_y",
          "center_of_gaus_fit_RA", "center_of_gaus_fit_DEC", "fit_x_axis", "fit_y_axis", "fit_theta",
          "deconv_x", "deconv_y", "integrated_intensity_fit", "ratio_residual"
         ]
catalogue = pd.read_csv(writeable + "catalogue_v2/part-00000", sep=",", names=header)
catalogue.head()

,label,total_pixels,x_pixels,y_pixels,integrated_intensity,brightest_pixel,brightest_pixel_x,brightest_pixel_y,brightest_pixel_RA,brightest_pixel_DEC,...,center_of_gaus_fit_y,center_of_gaus_fit_RA,center_of_gaus_fit_DEC,fit_x_axis,fit_y_axis,fit_theta,deconv_x,deconv_y,integrated_intensity_fit,ratio_residual
0,P244+48_0,28,4,6,0.658204,0.000776,139,5389,247.218439,47.945955,...,5389.424096,247.218388,47.946133,3.229095,1.546223,-0.116897,6.466828,0.000000,0.666204,0.116074
1,P244+48_1,16,4,4,0.362765,0.000715,164,5608,247.207640,48.037513,...,5608.597502,247.208414,48.037744,1.368732,1.592572,0.174488,0.000000,0.000000,0.373678,0.171477
2,P244+48_2,252,17,19,89.688294,0.070113,178,5618,247.199134,48.041879,...,5617.918421,247.199514,48.041837,1.987786,2.394094,-0.971472,2.431176,3.972809,83.919891,0.093799
3,P244+48_3,16,2,6,0.325570,0.000646,177,5680,247.201103,48.067684,...,5677.000000,247.200414,48.066449,1.929362,2.741847,-0.459429,2.154432,5.068247,0.336121,0.148926
4,P244+48_4,35,6,10,0.871635,0.000914,186,5485,247.191275,47.986609,...,5484.814843,247.190786,47.986543,3.308711,1.257191,0.423073,6.686271,0.000000,0.866828,0.260645
